In [1]:
import numpy as np
import pandas as pd
import os

from flipside import Flipside
flipside = Flipside("023b83a2-e411-4849-a961-3742c87bc9c9", "https://api-v2.flipsidecrypto.xyz")

We first set the name of the liquidity pool we want to query data from.

In [2]:
pool_name = "WETH-USDT 3000 60"

This write the sql code and start query.

In [3]:
sql = """
SELECT pool_name, tx_hash, action, block_number, block_timestamp, 
       liquidity_provider, nf_token_id,
       amount0_adjusted, token0_symbol, amount0_usd,
       amount1_adjusted, token1_symbol, amount1_usd,
       tick_lower, tick_upper
FROM ethereum.uniswapv3.ez_lp_actions
""" +\
" WHERE pool_name = '" + pool_name + "' " +\
"""
ORDER BY block_number DESC
"""

# Since there is a 30mb rate limit for each page, we need to split the data in seperate pages, we will merge them later
# If we are retrieving a dataset with more columns, we might want to set page_size to a lower number
query_result_set = flipside.query(sql, page_number = 1, page_size = 30000)

Here, we run a loop to merge the pages into one full dataset.

In [4]:
current_page_number = 1
page_size = 30000
# We first set total_pages to 2 (any number larger than current_page_number will do) for the loop to run, we will retrieve the actual total pages in the loop
total_pages = 2

# All the data will be stored in a list called "all_rows"
all_rows = []

while current_page_number <= total_pages:
  results = flipside.get_query_results(
    query_result_set.query_id,
    page_number=current_page_number,
    page_size=page_size
  )

  total_pages = results.page.totalPages
  if results.records:
      all_rows = all_rows + results.records
  
  current_page_number += 1

# Merge all rows to a dataframe
df = pd.DataFrame(all_rows)
df.drop(columns = ['__row_index'], inplace = True)
df.head(10)

,pool_name,tx_hash,action,block_number,block_timestamp,liquidity_provider,nf_token_id,amount0_adjusted,token0_symbol,amount0_usd,amount1_adjusted,token1_symbol,amount1_usd,tick_lower,tick_upper
0,WETH-USDT 3000 60,0x61e4c6c235334d7db55f6464798b6df5571e40fb8dc2...,DECREASE_LIQUIDITY,17763556,2023-07-24T14:27:59.000Z,0x41e420311ca4226d00157495ab014a262092aba3,NaN,0.000000,WETH,0.000000e+00,0.000000e+00,USDT,0.000000e+00,-208020,-194820
1,WETH-USDT 3000 60,0x849b04384bfbf191ff3aca380c78c8ddb59de458d224...,DECREASE_LIQUIDITY,17763085,2023-07-24T12:53:23.000Z,0x4a739ab9ddeb08fb04d77c5a0f68fc20657fdf9e,NaN,0.000000,WETH,0.000000e+00,0.000000e+00,USDT,0.000000e+00,-202560,-199200
2,WETH-USDT 3000 60,0xd9ab658711e217eb2be84737db527669fe83a3565345...,DECREASE_LIQUIDITY,17762833,2023-07-24T12:02:11.000Z,0x321da89db6993ee78394d5b436e7a2a3cea506df,319501.0,0.037327,WETH,6.910109e+01,1.519060e+02,USDT,1.518895e+02,-211080,-197160
3,WETH-USDT 3000 60,0xd9ab658711e217eb2be84737db527669fe83a3565345...,DECREASE_LIQUIDITY,17762833,2023-07-24T12:02:11.000Z,0x321da89db6993ee78394d5b436e7a2a3cea506df,NaN,0.000000,WETH,0.000000e+00,0.000000e+00,USDT,0.000000e+00,-211080,-197160
4,WETH-USDT 3000 60,0xc467b5f869fea61ceb66057edaceec3203ca5314f9cd...,INCREASE_LIQUIDITY,17762153,2023-07-24T09:44:59.000Z,0x947cf218f205d4c9ccb115a5a835647780fb7292,507827.0,0.020612,WETH,3.803231e+01,1.000000e+02,USDT,9.999511e+01,-414480,-191220
5,WETH-USDT 3000 60,0xb3f2e68c9f7c2819d70f985ad241de254c0ae3b692a2...,INCREASE_LIQUIDITY,17761055,2023-07-24T06:03:35.000Z,0xdab48129e631b3f9e5885bcd49e2b298c4c115eb,541080.0,0.001000,WETH,1.870625e+00,0.000000e+00,USDT,0.000000e+00,-199560,-196800
6,WETH-USDT 3000 60,0x04ee281b70f0f58c6bbc936c76d9c18398ee0ebf779b...,DECREASE_LIQUIDITY,17760830,2023-07-24T05:18:35.000Z,0x86c4e7bac249275376bc4e500757c1d4196ca77a,242178.0,290.579775,WETH,5.438180e+05,4.811634e+05,USDT,4.811145e+05,-207420,-193560
7,WETH-USDT 3000 60,0xfb1f48956212ad457f6daf7d1e26e02de08693be06e4...,DECREASE_LIQUIDITY,17760551,2023-07-24T04:21:59.000Z,0xe93854c13d1ac09928e1cbbf5f01b31f27092cf8,517058.0,0.053590,WETH,1.002676e+02,1.011242e+02,USDT,1.011128e+02,-207960,-194100
8,WETH-USDT 3000 60,0xcffaf3560b3c86843f9db759b0db1b1b4cd7e8a04b5e...,DECREASE_LIQUIDITY,17759651,2023-07-24T01:20:59.000Z,0x9431bb66f5be84beee17348dd1a3cd07340ebe74,540344.0,0.011087,WETH,2.082860e+01,1.769994e+01,USDT,1.769869e+01,-201480,-200220
9,WETH-USDT 3000 60,0xb55bc3010f7011e2a3bb229a97d3159df448d30d9022...,DECREASE_LIQUIDITY,17758604,2023-07-23T21:50:23.000Z,0x36a454aef52938c8637cd4689b2980c1cfd43389,NaN,5599.063349,WETH,1.057109e+07,6.026887e+06,USDT,6.024441e+06,-200940,-200880


Lastly, we want to save the data in our desire repository.

In [5]:
df.to_csv("/Users/zhicong/Dropbox/DEXs/uniswapv3_lp_actions/" + pool_name + ".csv", index=False)